# Evaluative Routing

One of the most interesting applications of JSON outputs in agentic development is *routing* the workflow to different actions depending on the LLM's assessment.

Prompt a model with a collection of context and criteria and then ask it to create a decision. That decision can then be parsed as JSON and used to shape complex, versatile systems.

As routing becomes increasingly elaborate, we begin to see the border between *workflows* and *agents* grow clearer. Remember, we've defined 'autonomous agents' as AI systems that decide their own path (and the number of steps it will take) to accomplish a given task. A particularly sophisticated type of routing that leads to autonomous behavior is something we might call *evaluative routing*—where the model evaluates the output it produces and decides if it needs more work or if it's ready to be handed back to the user.

By making decisions based on qualitative evaluations, our agentic system can begin behaving like a totally unsupervised agent. Let's see it in practice.

## Setup

First, let's import the necessary libraries and set up our connection to the OpenAI API:

▶ Jupyter Help

In [ ]:
from openai import OpenAI
import json

client = OpenAI()

def get_completion(prompt, system_prompt="You are a helpful assistant.", json_mode=False):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": prompt}],
        response_format={"type": "json_object"} if json_mode else None
    )
    return response.choices[0].message.content

Now, take a look at the following cell. In it, we use JSON mode to instruct the model to route a customer service request to either a human or a chatbot depending on a set of criteria. Try changing the customer support request in the `request` variable to a few different types of queries to see how the model behaves.

In [ ]:
def route_customer(request):
    system_prompt = "You're a helpful assistant that outputs JSON. Do not include any markdown backticks in your response."
    prompt = f"""
    <instructions>Below you'll see a customer service request.
    Your job is to decide whether the request should be routed to a human or a chatbot.
    You should also provide a reason for your decision.
    The request is:
    {{request}}
    </instructions>
    <format>
    Your response should have two fields:
    - "decision": "human" or "chatbot"
    - "reason": a short explanation for your decision

    Route to a human if:
    - The request involves a complaint or dissatisfaction
    - The request requires account changes or financial transactions
    - The request mentions legal issues or threatens escalation

    Route to a chatbot if:
    - The request is for basic information or FAQs
    - The request is for simple status updates
    - The request is for general product information

    For instance:
    {{
        "decision": "human",
        "reason": "The request is a complex issue that requires a human touch."
    }}
    </format>
    """
    response = get_completion(prompt, system_prompt, json_mode=True)
    return response

request = "I need help with my account."
decision = route_customer(request)
json_decision = json.loads(decision)
decision = json_decision['decision']
reason = json_decision['reason']

print("Request:", request)
print("Decision: ", decision)
print("Reason: ", reason)

### Checkpoint 1/3

Now, assuming you've got the `decision` and `reason` variables accessible from the last cell, let's write a little code that routes the LLM's decision to two different outcomes. We've defined two placeholder functions `handle_by_chatbot` and `handle_by_human`, which should be called depending on the content of the `decision` variable.

In the second half of the cell, write a control flow that calls `handle_by_chatbot` if `decision` is equal to `"chatbot"` and `handle_by_human` if `decision` is equal to `"human"`.